In [21]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
import numpy as np
import base64
from collections import defaultdict

def ecb(plain, key):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()
    pad = padding.PKCS7(128).padder()
    return aes.update(pad.update(plain) + pad.finalize()) + aes.finalize()

def pkcs7(b, size=128):
    bsz = len(b)
    size //= 8
    sz = size * (bsz // size + 1)
    return b + bytes([sz - bsz]) * (sz - bsz)

def uses_ecb(cipher):
    counts = defaultdict(int)

    encrypted = cipher(b'A' * 52)

    for blk in np.frombuffer(encrypted, 'u1').reshape(-1, 16):
        counts[blk.tobytes()] += 1
    
    return np.any(np.array(list(counts.values())) > 1)

In [22]:
K = np.random.bytes(16)
S = b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK'


In [23]:
def encryption_oracle(plain, key=K, secret=S):
    plain = pkcs7(plain + base64.decodebytes(secret))
    return ecb(plain, key)


In [24]:
def cipher_blksz(cipher):
    p = b""
    e = cipher(p)
    s = len(e)
    while s == len(e):
        p += b'A'
        e = cipher(p)
    return len(e) - s


In [25]:
cipher_blksz(encryption_oracle)

16

In [26]:
uses_ecb(encryption_oracle)

True

In [27]:
def sb_ecb_break(cipher):
    """Single-Byte (Byte-at-a-time) ECB decryption (Simple: no prefix)"""

    blksz = cipher_blksz(cipher)

    if not uses_ecb(cipher):
        return -1

    tgtsz = len(cipher(b""))

    plain = b'A' * blksz

    for i in range(tgtsz):
        crafted = b'A' * (blksz - (i % blksz) - 1)
        crafted = cipher(crafted)
        crafted = [crafted[i:i+blksz] for i in range(0, len(crafted), blksz)]

        brute = [cipher(plain[-(blksz - 1):] + bytes([i])) for i in range(128)]
        brute = [[b[i:i+blksz] for i in range(0, len(b), blksz)] for b in brute]
        
        for j, b in enumerate(brute):
            if crafted[i // blksz] == b[0]:
                plain += bytes([j])
                break
    
    return plain[16:]


In [28]:
print(sb_ecb_break(encryption_oracle).decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



In [29]:
print(base64.decodebytes(S).decode())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

